In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

llm = init_chat_model("gemini-2.5-pro",model_provider="google_genai", temperature=0)
response = llm.invoke( 'hola soy cesar y mi numero es 123456789' )
response.pretty_print()

================================== Ai Message ==================================

¡Hola, César! Mucho gusto.

Gracias por la información, pero es muy importante que sepas algo sobre cómo funciono: como inteligencia artificial, no tengo memoria de nuestras conversaciones y **no guardo datos personales** como tu nombre o tu número.

De hecho, por tu propia seguridad y privacidad, te recomiendo **no compartir información sensible** como esta en internet.

Ahora que sabes esto, ¿en qué puedo ayudarte hoy?


In [ ]:
response = llm.invoke( 'cual es el clima de la ciudad de tijuana?')
response.pretty_print()

In [26]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar productos adecuados según sus necesidades.
y tus productos son los siguientes:
- Computadora
- mouse
- teclado
- audifionos
- mauspad
"""
message = [
  ("system", system_prompt),
  ("user", "Dime los productos que tienes para vender")
]

response = llm.invoke(message)
response.pretty_print()

================================== Ai Message ==================================

¡Hola! Claro que sí. Con gusto te ayudo.

Estos son los productos que tenemos disponibles:

*   **Computadora**
*   **Mouse**
*   **Teclado**
*   **Audífonos**
*   **Mauspad** (Mousepad)

¿Estás buscando armar un equipo completo desde cero o necesitas reemplazar o añadir algún accesorio?

Cuéntame un poco más sobre lo que necesitas, ¿es para trabajar, para estudiar, para jugar? Así puedo darte la mejor recomendación. ¡Estoy para ayudarte


In [ ]:
from langchain_core.tools import tool
import requests
from typing import Optional
# El "Razonamiento" del Modelo: El modelo (Gemini 2.5 Pro) es lo suficientemente inteligente como para analizar la situación:

# "El usuario quiere la lista de productos."
# "Tengo una herramienta llamada get_products para eso."
# "Pero... la herramienta get_products me exige un argumento price que no tengo."
# "Como no puedo llamar a la herramienta sin el argumento requerido, no la llamaré. En su lugar,
#  responderé con texto, probablemente pidiendo más información o dando una respuesta general."
@tool("get_products", description="Get the products that the store offers and filter by price.")
def get_products(price: Optional[int] = None):
	"""Get the products from the Fake Store API."""
	response = requests.get("https://api.escuelajs.co/api/v1/products")
	products = response.json()

	return "".join([f"{product['title']} - ${product['price']}" for product in products])

In [32]:
get_products.invoke({"price": 100})

'Artss - $11122Artix - $123123Alpha1760905210304 - $100Furniture product_RHWV - $10Furniture product - $10Furniture product_9L7X - $10Furniture product_JVHW - $10Furniture product_SANL - $10Chairs product_OT1Z - $12aabxdc - $1110'

In [29]:
@tool("get_weather", description="Get the weather of a city")
def get_weather(city: str):
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    data = response.json()
    response = f"The weather in {city} is {data["current_weather"]["temperature"]}C with {data["current_weather"]["windspeed"]}km/h of wind."
    return response

get_weather.invoke({"city": "tijuana"})

'The weather in tijuana is 27.4C with 13.4km/h of wind.'

In [33]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofreces en la tienda")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_products',
  'args': {},
  'id': 'b1a6f7c8-7f66-4932-a956-0d07bfd07b85',
  'type': 'tool_call'}]

In [34]:
messages = [
    ("system", system_prompt),
    ("user", "Hola, que tal?")
]
response = llm_with_tools.invoke(messages)
response.text

'¡Hola! ¿Cómo estás? Soy tu asistente de ventas. ¿En qué puedo ayudarte hoy? Puedo buscar productos o darte el pronóstico del tiempo.'

In [37]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que necesitan y dar el clima de la ciudad

Tus tools son:
- get_products: para obtener los productos que ofreces en la tienda
- get_weather: para obtener el clima de la ciudad
"""
messages = [
    ("system", system_prompt),
    ("user", "Cual es el clima en la capital de baja california?")
]
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_weather',
  'args': {'city': 'Mexicali'},
  'id': '368bb953-7705-478f-b785-db3e98206ac0',
  'type': 'tool_call'}]